In [ ]:
import os
import random

import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
import albumentations.pytorch as Apy

import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader,random_split
from torch.nn import functional as F


In [ ]:
train_csv = pd.read_csv("drive/MyDrive/Colab Notebooks/train.csv")
train_csv.head()

In [ ]:
os.mkdir("drive/MyDrive/Colab Notebooks/training_images")
for i in list(set(train_csv["label"])):
    os.mkdir("drive/MyDrive/Colab Notebooks/training_images/{}".format(i))

In [ ]:
# 이미지 라벨링을 위한 폴더별로 이미지 넣기
for file in tqdm(all_training_files):
    # 이미지별 라벨달기
    label = int(train_csv[train_csv["image_id"] == file]["label"])
    dirname = "drive/MyDrive/Colab Notebooks/train_images/" + file
    outname = "drive/MyDrive/Colab Notebooks/training_images/{}/{}".format(label, file)
    shutil.copyfile(dirname, outname)

In [ ]:
# 데이터 전처리 및 downloader을 이용한 데이터 셋 만들기
data_dir = "drive/MyDrive/Colab Notebooks/training_images/"
batch_size = 8
def get_data_loaders(data_dir, batch_size):
    transform = transforms.Compose([transforms.Resize((220, 220)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation(20),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
    all_data = datasets.ImageFolder(data_dir, transform=transform)  
    train_data_len = int(len(all_data)*0.75)
    valid_data_len = int(len(all_data) - train_data_len)
    train_data, val_data = random_split(all_data, [train_data_len, valid_data_len])
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
    return ((train_loader, val_loader), all_data.classes)

In [ ]:
get_data_loaders(data_dir, batch_size)
(train_loader, val_loader), classes = get_data_loaders(data_dir, 64)

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(14, 10))
for idx in np.arange(20):
    ax = fig.add_subplot(5, 12/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])

In [ ]:
#현재 토치사이즈 보기
imgs = 0
for n, (img, labels) in enumerate(train_loader):
    print(n, img.shape, labels.shape)
    imgs = img
    break

In [ ]:
#네트웨크 만들기
import torch.nn.functional as F
from torch.autograd import Variable

class my_network(nn.Module):
    def __init__(self):
        super(my_network, self).__init__()
        self.conv1 = nn.Conv2d(3,64,5)
        self.conv2 = nn.Conv2d(64,30,5) 
        self.fc1 = nn.Linear(30*5*5, 5)
        self.fc2 = nn.Linear(5, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x), inplace=True)
        x = F.max_pool2d(x,(2,2))
        x = F.relu(self.conv2(x), inplace=True)
        x = F.max_pool2d(x,(2,2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)
        return x  

my_net = my_network()

In [ ]:
#손실함수 정의
optim = torch.optim.SGD(my_net.parameters(), lr=0.001, momentum=0.9)
loss_function = nn.CrossEntropyLoss()

In [ ]:
#훈련하기
epoch_num = 3
for epoch in range(epoch_num):
    for i, data in enumerate(train_loader):
        inputs, labels = data

        inputs, labels = Variable(inputs), Variable(labels)

        optim.zero_grad()
        out = my_net(inputs)
        loss = loss_function(out, labels)
        loss.backward()
        optim.step()

    if i % 64 == 0:
        print("%d => loss : %.3f" %(i, loss))

print("train over")


In [ ]:
#예측하기
total = 0
correct = 0
for data in val_loader:
    images, labels = data
    outputs = my_net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the test images: %f'%(100*correct/total))

In [ ]:
#GPU 연결
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model.to(device)